In [75]:
import gspread

from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd

# Définition des informations d'identification
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(r"/content/drive/MyDrive/apikey.json", scope)

# Autorisation et ouverture de la feuille de calcul
client = gspread.authorize(credentials)
spreadsheet = client.open('BUDGET')
worksheet = spreadsheet.worksheet('all_combine')

# Récupération des données et création de la DataFrame
data = worksheet.get_all_values()
headers = data.pop(0)  # Retirer la première ligne et l'assigner à headers
budget = pd.DataFrame(data, columns=headers)


In [76]:
budget

,DATE/MOIS,LIBELLE,MONTANT,MOYEN DE PAIEMENT
0,16/03/2020,CARREFOUR SURESNES VERDUN,19.53 €,CB
1,19/03/2020,STIS MULTIMARKET,22.53 €,CB
2,19/03/2020,PHAMARCIE,4.36 €,CB
3,19/03/2020,FRANPRIX,7.63 €,TR
4,24/03/2020,CARREFOUR SURESNES VERDUN,99.15 €,CB
...,...,...,...,...
497,,PLATS CHINOIS,22.00€,ESP
498,25/08/2023,CARREFOUR,190.00€,CB BOURSO FLO
499,,,123.50€,CB BOURSO FLO
500,,TGTG,12.00€,CB BOURSO FLO


In [77]:
budget = budget.replace('', None)

In [78]:
budget.rename(columns={"DATE/MOIS": "date", "LIBELLE": "place","MOYEN DE PAIEMENT":"paiment_method", "MONTANT ":"amount"},inplace = True)

In [79]:
budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            340 non-null    object
 1   place           432 non-null    object
 2   amount          502 non-null    object
 3   paiment_method  497 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB


In [80]:
#fill null data with previous data
budget.ffill(inplace=True)

In [81]:
budget

,date,place,amount,paiment_method
0,16/03/2020,CARREFOUR SURESNES VERDUN,19.53 €,CB
1,19/03/2020,STIS MULTIMARKET,22.53 €,CB
2,19/03/2020,PHAMARCIE,4.36 €,CB
3,19/03/2020,FRANPRIX,7.63 €,TR
4,24/03/2020,CARREFOUR SURESNES VERDUN,99.15 €,CB
...,...,...,...,...
497,19/08/2023,PLATS CHINOIS,22.00€,ESP
498,25/08/2023,CARREFOUR,190.00€,CB BOURSO FLO
499,25/08/2023,CARREFOUR,123.50€,CB BOURSO FLO
500,25/08/2023,TGTG,12.00€,CB BOURSO FLO


In [82]:
budget['date'] = pd.to_datetime(budget['date'],dayfirst=True)
budget['month'] = budget['date'].dt.month
budget['year'] = budget['date'].dt.year

In [83]:
#defining the cleaning price function
def clean_amount (amount):
  amount = float(amount.replace('€', '').replace(',', ''))
  return amount

#applying the function to the price column of df_calendar
budget['amount'] = budget['amount'].apply(lambda x: clean_amount(x))

#transforming the column as a float
budget['amount'] = budget['amount'].astype(float)

In [84]:
budget

,date,place,amount,paiment_method,month,year
0,2020-03-16,CARREFOUR SURESNES VERDUN,19.53,CB,3,2020
1,2020-03-19,STIS MULTIMARKET,22.53,CB,3,2020
2,2020-03-19,PHAMARCIE,4.36,CB,3,2020
3,2020-03-19,FRANPRIX,7.63,TR,3,2020
4,2020-03-24,CARREFOUR SURESNES VERDUN,99.15,CB,3,2020
...,...,...,...,...,...,...
497,2023-08-19,PLATS CHINOIS,22.00,ESP,8,2023
498,2023-08-25,CARREFOUR,190.00,CB BOURSO FLO,8,2023
499,2023-08-25,CARREFOUR,123.50,CB BOURSO FLO,8,2023
500,2023-08-25,TGTG,12.00,CB BOURSO FLO,8,2023


In [85]:
budget['paiment_method'].value_counts()

TR                              80
ESP                             75
CB CJ                           59
CB                              41
CB RX                           33
CB*                             28
CB FLO                          28
CB CJ                           21
CB                               9
TR RX                            8
ESP*                             8
ESP+TR                           7
 CB RX BOURS (CARTE CADEAUX)     7
CB BOURS RX                      7
CB BOURSO RX                     6
TR*                              6
WECHAT PAY                       6
CAGNOTTE                         6
CB BOURSO FLO                    6
CB BOURS FLO                     6
TR+ESP                           5
TR+CB                            4
TR FLO                           3
TR +ESP                          3
CB RX HB                         3
CB+ESP                           2
ESP+CB                           2
ESP + TR                         2
ESP+CB FLO          

In [86]:
#budget['paiment_method'] = budget['paiment_method'].replace(r'(.*CB.*RX|Ruixue|ruixue.*)', 'Carte Ruixue', regex=True).replace(r'(.*CB.*FLO.*)', 'Carte Flo', regex=True).replace(r'(.*CB.*CJ.*)', 'Carte compte-joint', regex=True).replace(r'.*CB.*', 'Carte indéterminée', regex=True).str.replace('^ESP.*','Espèce').str.replace('^TR.*','Ticket resto').str.replace('^WECHAT.*','Wechat').replace('GAGNOTTE','Cagnotte').replace('CAGNOTTE','Cagnotte')

In [87]:
budget['paiment_method'] = budget['paiment_method'].replace(r'.*CB.*', 'Carte', regex=True)\
    .replace('^ESP.*','Espèce', regex=True)\
    .replace('^TR.*','Ticket resto', regex=True)\
    .replace('^WECHAT.*','Wechat', regex=True)\
    .replace(['GAGNOTTE', 'CAGNOTTE'],'Cagnotte')

In [88]:
budget['place'].value_counts().head(20)

CARREFOUR SURESNES VERDUN    49
CASINO SUPERMARCHE           24
CARREFOUR                    24
Casino supermarché           23
CARREFOUR MONTESSON          19
MARCHE                       19
CARREFOUR DRIVE              18
Carrefour Montesson          16
UBEREATS HEISENBURGER        10
MARCHE+BOUCHERIE              9
BOULANGERIE                   9
SUPER U                       9
GRAND FRAIS                   8
MONOPRIX PLUS                 8
UBER EATS Breton burger       8
CASINO                        8
CARREFOUR CITY                6
HELLO FRESH                   6
MON MARCHE                    6
Ubereats Breton burger        5
Name: place, dtype: int64

In [89]:
import re

pattern_carref = re.compile(r'.*carref.*', re.IGNORECASE)
pattern_casino = re.compile(r'.*CASINO.*', re.IGNORECASE)
pattern_ubereats = re.compile(r'.*UBER.*', re.IGNORECASE)

budget['place'] = budget['place'].str.replace(pattern_carref, 'Carrefour')\
    .str.replace(pattern_casino, 'Casino')\
    .str.replace(pattern_ubereats, 'Ubereats')

In [90]:
import plotly.express as px

fig = px.bar(budget,x='paiment_method',y='amount')
fig

In [91]:
budget_monthly= budget.groupby('date',as_index=False).sum()
budget_monthly

<ipython-input-91-1b7a7a4f130d>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,date,amount,month,year
0,2020-03-16,19.53,3,2020
1,2020-03-19,34.52,9,6060
2,2020-03-24,118.15,6,4040
3,2020-03-28,13.80,3,2020
4,2020-03-31,13.80,3,2020
...,...,...,...,...
320,2023-08-08,135.00,16,4046
321,2023-08-12,50.00,8,2023
322,2023-08-19,87.00,24,6069
323,2023-08-25,325.50,24,6069


In [92]:
fig2 = px.line(budget_monthly,x='date',y='amount')
fig2

In [93]:
budget.to_csv('budget_clean.csv', index=False)

In [94]:
# Définition des informations d'identification
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(r"/content/drive/MyDrive/apikey.json", scope)

# Autorisation et ouverture de la feuille de calcul
client = gspread.authorize(credentials)
spreadsheet = client.open('BUDGET')
worksheet = spreadsheet.worksheet('common')

# Récupération des données et création de la DataFrame
data = worksheet.get_all_values()
headers = data.pop(0)  # Retirer la première ligne et l'assigner à headers
common = pd.DataFrame(data, columns=headers)

In [95]:
common.head(10)

,date,brand,amount,where,paiment_method
0,2020-11-7,Matelas Bellecour,479.99 €,Internet,Paypal flo
1,2020-11-9,Auchan,40.00 €,Internet,CB RX B
2,2020-11-12,Notaire,"1,537.66 €",,Chèque flo
3,2020-12-13,Bouygues,58.00 €,Internet,CB cj
4,2020-11-14,Leroy merlin,173.70 €,Internet,CB cj
5,,,146.08 €,magasin,CB cj
6,2020-11-15,Leroy merlin,10.50 €,MAGASIN,CB cj
7,2020-11-17,Auchan,42.99 €,Internet,CB RX B
8,2020-11-20,Auchan,36.00 €,Internet,CB RX B
9,2020-11-21,Leroy merlin,116.05 €,MAGASIN,CB CJ


In [96]:
common = common.replace('', None)

In [97]:
# Spécifier les colonnes à remplir vers l'avant (frontfill)
columns_to_fill = ['date', 'brand', 'paiment_method']

# Effectuer le remplissage vers l'avant sur les colonnes spécifiées
common[columns_to_fill] = common[columns_to_fill].fillna(method='ffill')

#drop les colonnes sans prix car impossible
common.dropna(subset = 'amount',inplace = True)

#rempli les null dans where par Other
common['where'].fillna('Other',inplace=True)

#applying the function to the price column of df_calendar
common['amount'] = common['amount'].apply(lambda x: clean_amount(x))

#transforming the column as a float
common['amount'] = common['amount'].astype(float)

In [98]:
common.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 88
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            74 non-null     object 
 1   brand           74 non-null     object 
 2   amount          74 non-null     float64
 3   where           74 non-null     object 
 4   paiment_method  74 non-null     object 
dtypes: float64(1), object(4)
memory usage: 3.5+ KB


In [99]:
common['paiment_method'].value_counts()

CB CJ                          24
PAYPAL CJ                       9
CB FLO                          7
CB RX B                         5
CB cj                           4
CB RX                           3
CB RX H                         2
ESP                             2
CB CJ                           2
Chèque flo                      2
CB RX BOURSO                    1
CB BOURSO+CB FLO+CJ             1
CB FLO BOURSO                   1
CB BOURSO(CARTE CADEAUX+ESP     1
CB BOURSO RX                    1
CHEQUE FLO                      1
Paypal flo                      1
CB CJ+CB RX                     1
CB RX BOUSO+CB FLO BOURSO       1
CB RX BINANCE                   1
CB BN RX                        1
CHEQUE                          1
CB RXH                          1
CAGNOTTE                        1
Name: paiment_method, dtype: int64

In [100]:
# Définir le dictionnaire de correspondances avec les expressions régulières
correspondances = {
    r'.*CB.*CJ.*': 'carte compte-joint',
    r'.*PAYPAL.*': 'paypal',
    r'.*CB.*FLO.*': 'carte flo',
    r'.*CB.*RX.*': 'carte ruixue',
    r'.*CAGNOTTE.*': 'cagnotte',
    r'.*ESP.*':'Espèce',
    r'.*chèque|cheque.*' : 'chèque'
}

# Fonction de remplacement basée sur les correspondances
def remplacer_valeur(valeur):
    for motif, correspondance in correspondances.items():
        if re.match(motif, valeur, flags=re.IGNORECASE):
            return correspondance
    return autre

# Appliquer la fonction de remplacement à la colonne 'paiment_method'
common['paiment_method'] = common['paiment_method'].apply(remplacer_valeur)

In [101]:
common['where'].value_counts()

Internet    36
INTERNET    18
MAGASIN     14
Other        5
magasin      1
Name: where, dtype: int64

In [102]:
common['where'] = common['where'].replace('INTERNET','Internet').replace('MAGASIN','Magasin').replace('magasin','Magasin')


In [103]:
common

,date,brand,amount,where,paiment_method
0,2020-11-7,Matelas Bellecour,479.99,Internet,paypal
1,2020-11-9,Auchan,40.00,Internet,carte ruixue
2,2020-11-12,Notaire,1537.66,Other,chèque
3,2020-12-13,Bouygues,58.00,Internet,carte compte-joint
4,2020-11-14,Leroy merlin,173.70,Internet,carte compte-joint
...,...,...,...,...,...
84,2023-4-1,LE WAGON,4305.00,Other,carte flo
85,2023-4-6,CARREFOUR,629.40,Internet,carte ruixue
86,2023-4-15,CARREFOUR,109.00,Internet,cagnotte
87,2023-5-13,JO 2024,568.40,Internet,carte ruixue


In [104]:
# Définition des informations d'identification
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(r"/content/drive/MyDrive/apikey.json", scope)

# Autorisation et ouverture de la feuille de calcul
client = gspread.authorize(credentials)
spreadsheet = client.open('BUDGET')
worksheet = spreadsheet.worksheet('CHATCHAT')

# Récupération des données et création de la DataFrame
data = worksheet.get_all_values()
headers = data.pop(0)  # Retirer la première ligne et l'assigner à headers
cat = pd.DataFrame(data, columns=headers)

In [105]:
cat.head()

,date,where,amount,comment,category
0,2021,,,,
1,2021-9-30,ZOOPLUS,53.01 €,,food
2,2021-10-10,ZOOPLUS,108.59 €,,food
3,,ASSOCIATION EMA,200.00 €,,association
4,2021-10-12,ZOOPLUS,62.94 €,,food


In [106]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      38 non-null     object
 1   where     38 non-null     object
 2   amount    38 non-null     object
 3   comment   38 non-null     object
 4   category  38 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB


In [107]:
cat


,date,where,amount,comment,category
0,2021,,,,
1,2021-9-30,ZOOPLUS,53.01 €,,food
2,2021-10-10,ZOOPLUS,108.59 €,,food
3,,ASSOCIATION EMA,200.00 €,,association
4,2021-10-12,ZOOPLUS,62.94 €,,food
5,,AMAZON,18.08 €,,jouet
6,2021-10-20,VETERINAIRE,90.00 €,,soin
7,2021-11-1,ZOOPLUS,14.04 €,,food
8,2021-11-11,ZOOPLUS,56.44 €,,food
9,,FONTAINE,90.00 €,,jouet


In [108]:
cat = cat.replace('', None)

In [109]:
cat['date'] = pd.to_datetime(cat['date'],dayfirst=True)

In [110]:
#dropping any columns without a name
cat.drop(cat.columns[cat.columns ==''],axis=1,inplace=True)

#dropping any row with a missing amount
cat.dropna(subset=['amount'],inplace=True)

#front filling date
cat['date'].ffill(inplace=True)

#applying the function to the price column of df_calendar
cat['amount'] = cat['amount'].apply(lambda x: clean_amount(x))

#transforming the column as a float
cat['amount'] = cat['amount'].astype(float)

In [111]:
cat

,date,where,amount,comment,category
1,2021-09-30,ZOOPLUS,53.01,None,food
2,2021-10-10,ZOOPLUS,108.59,None,food
3,2021-10-10,ASSOCIATION EMA,200.00,None,association
4,2021-10-12,ZOOPLUS,62.94,None,food
5,2021-10-12,AMAZON,18.08,None,jouet
6,2021-10-20,VETERINAIRE,90.00,None,soin
7,2021-11-01,ZOOPLUS,14.04,None,food
8,2021-11-11,ZOOPLUS,56.44,None,food
9,2021-11-11,FONTAINE,90.00,None,jouet
10,2021-12-12,VETERINAIRE,77.00,None,soin


In [112]:
import pandas as pd
import pandas_gbq

# Define your project ID and BigQuery table name
project_id = 'dev-acolyte-384710'
table_budget = 'home_expenses.budget'
table_common = 'home_expenses.common'
table_cat = 'home_expenses.cat'


# Export the DataFrame to BigQuery
pandas_gbq.to_gbq(common, table_common, project_id=project_id, if_exists='replace')
pandas_gbq.to_gbq(cat, table_cat, project_id=project_id, if_exists='replace')
pandas_gbq.to_gbq(budget, table_budget, project_id=project_id, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 7423.55it/s]


In [113]:
budget.tail(1)

,date,place,amount,paiment_method,month,year
501,2023-09-07,Carrefour,12.6,Carte,9,2023
